# Supplementary Labsheet 1: Interrupts and Timers

This worksheet will cover Interrupts and Timers, which are internal devices to the microcontroller.  You could think of them as independent and very simple 'computers' inside the microcontroller.  

In this labsheet we are going to:
 - Set up Timer3 to flash an LED every 0.5 seconds.
 - Experiment with how an interrupt behaves relative to the loop() routine.

The worksheet goes right down into register operations for the microcontroller.  As a rule, you can think of registers as 'tables of switches'.  To utilise or configure some hardware within the microcontroller, you need to find out the right place to put a 1 or 0 to switch something on or off.

Do not worry if you find this overwhelming - everyone does at first because there is too much information to learn at once.  It takes time to get used to all the strange names and processes.  In fact, one of the necessary learning steps with microcontrollers is to get used to the huge manuals, and the intimidating feeling of searching for the right information.  With time, once you get used to it, it can become fun.  We will guide you through the process, and hopefully you will come to see that working with low level registers is a step-by-step process of investigation.  

You can't really break a microcontroller with just code on its own - so try things out, see what happens.  Microcontrollers are tricky to debug, so take small steps.

You may regard much of this labsheet as theory - so see this as an opportunity to explore.  In the core labsheets we give you code to setup the encoder pin change interrupts.  By following this labsheet, you should get closer to understand what is going on.  If you can get to grips with this labsheet, you may wish to try putting your PID update routines into a timer interrupt.  Another interesting use of the timer interrupt could be to read the line sensors.  

Beyond the study of this Unit you'll likely find and use a library of code written by someone else.  It is quite normal to search help forums and adjust someone elses code.  However it is very likely you'll need to understand how interrupts and timers work to satisfy your own requirements.  

You may want to open the <a href="http://ww1.microchip.com/downloads/en/devicedoc/atmel-7766-8-bit-avr-atmega16u4-32u4_datasheet.pdf">manual</a> for the chip we are using, and find the references as we go.





**Note**: This labsheet has a large amount of introductory text before you are introduced to a full template of code (section "Timer 3 Code Template").


<hr><br><br><br><br>





# Using _timer3_ to flash an LED every 0.5 seconds

In this exercise we are going to set up and use the peripheral Timer 3.  Once we have set it up, we can use the timer to create or manage frequent background tasks.  We might use a Timer to regularly check a sensor, drive motors, keep track of time, or other time-precise task.  You should recognise the following illustration from the accompanying lecture:

<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/Timer3_and_CPU.png?raw=true"/>

You might be asking, 'Why Timer 3?'.  The reasoning is that Timer 0 is used by the Arduino software to create _millis()_ functionality, and Timer 1 is used by the Arduino software to control _Servo Motors_ (we won't use these, but they are very useful, so we'll stay away from this timer), and Timer 4 is used to drive the buzzer - so we don't want to use either of these.  A bit more info is available <a href="https://www.pololu.com/docs/0J69/all#3.10">here</a>

Timer 3 is interesting because it can store 16 bit numbers, which means it can count up relatively high.  By counting high, we can configure it create very infrequent interrupts - so slow, in fact, that we can blink an LED slow enough to see it.  We are going to explore and understand how in this exercise.  

There are two parts to using and setting up a Timer:
- Write the **Interrupt Service Routine (ISR)**: the code which is called when the Timer flags an interrupt.
- Write the **Timer setup**: code to configure it to run at a time interval.

We will use Timer3 to blink an LED every 0.5 seconds.



<hr><br><br><br><br>

### A look at just the Interrupt Service Routine (ISR)

The following is a simple ISR.  It looks similar to a normal function block of code, although with a strange syntax:

```c
// Global variable to remember the
// on/off state of the LED.  
volatile boolean DEBUG_LED_STATE = false;

// The ISR routine.
// The name TIMER3_COMPA_vect is a special flag to the
// compiler.  It automatically associates with Timer3 in
// CTC mode.
ISR( TIMER3_COMPA_vect ) {

  // Invert LED state.
  DEBUG_LED_STATE = !DEBUG_LED_STATE;

  // Enable/disable LED
  digitalWrite(13, DEBUG_LED_STATE);

}
```

The above code is our _Interrupt Service Routine (ISR)_.  The ISR will get called automatically by _timer3_.  We have written this ISR to be very simple.  All it does it:

- The code changes the stored state of our LED in the variable *DEBUG_LED_STATE* .  
- The code then sets the LED pin 13 using _digitalWrite()_.

Importantly, the code uses a global variable, *DEBUG_LED_STATE* which has the keyword **volatile** before it.  **volatile** tells the compiler that it is likely the value of *DEBUG_LED_STATE* will change unexpectedly.  Using a Timer means that the compiler cannot predict when a variable will change.  Ordinarily, compilers attempt to optimise memory usage with some clever tricks.  Timer3 ISR will be changing the variable *DEBUG_LED_STATE* outside of the main sequence of the _loop()_ code, so we have to use **volatile** to prevent errors.  You can find out more about the volatile keyword <a href="https://www.arduino.cc/reference/en/language/variables/variable-scope--qualifiers/volatile/">here</a>.


These are some general rules about writing *ISR*'s:
- __It is important to note__: that the length of your ISR can badly effect the behaviour of your microcontroller, and create errors which are very difficult to debug.  Because an ISR happens in the background, it breaks ordinary sequential nature of your loop() code.


- __ISR are sensitive to time:__ It takes time for your microcontroller to actually execute a line of code.  Therefore, if your ISR has many lines of code, it is possible that your ISR will take longer to execute than the frequency that your timer is supposed to raise an interrupt flag.  On some microcontrollers, it is possible for your ISR to be interrupted by itself (a second copy of the ISR begins running, and then a third, forth, etc, until your microcontroller crashes)!  On this microcontroller, the interrupt flag is disabled while inside the ISR, so your ISR will effectively stop the timers clock, and you'll have lost your time-critical precision.  


- __What does this mean?__  In either case, it means your timer will no longer be working on time, which makes it useless.  


- __As a general rule__ always keep your ISR as short and as simple as possible.  

<hr><br><br><br><br>

## Setting Up Timer3

We have written our Interrupt Service Routine, above, so now we must write some code to set up Timer 3.  We need to tell the microcontroller how often we want the ISR to be run.  It is good practice to encapsulate the setup code into a function, and we will call this **setupTimer3()**.

If you have worked with high-level software, for example Java, it might seem like it should be as simple as telling the timer to be any frequency we like.  However, the speed of a micro-controller Timer is linked to the central clock, which is the 16Mhz clock of the 3Pi+ microcontroller. For a lot of applications this is too fast (16 million times a second!).  The electronics of the Timer peripheral can only roughly divide the 16Mhz clock down, which is imprecise.  Therefore, a second parameter, a value for the timer to count up to, provides a finer resolution.  

A timer has these two parts of set up because it allows it to be very flexible.  If we divide down the clock speed of the timer, we can **coarsely** make the interrupt very fast or very slow.  By changing the value it counts up to, we can **finely** select the exact frequency.  

You can imagine a Timer working in this way like waiting for a bath to fill up with water.  When the bath overflows, we run a piece of code (the ISR) and reset the bath back to empty to start again.  With this analogy, there are two main parameters we can change:
- how fast water flows (divided clock frequency).
- how big the bath tub is (value to count to).

We set how fast the water flows as a rough frequency (a division of 16Mhz), and then change the size of the bath to get an exact frequency (the count value).  

With micro-controllers, the size of the bath (count value) is usually limited to an 8 bit number (max 256) or 16 bit number (max 65536) - so we cannot simply change the size of the bath (count) in an unlimited way either.  We have to therefore balance the clock-division (rough timing) and count parameters (fine tuning).  

As a setup process, we first find a close division of the 16Mhz clock to our desired frequency, then determine a correct count value.

As an exercise, we want our timer3 to raise an interrupt every 0.5 seconds, which is every 500ms.  This is also known as 2 Hertz (2 times a second).  Note how exceptionally slow 2Hz is compared to 16Mhz!  We will therefore have to divide the 16Mhz clock down - called **prescaling**, and the division is known as the **prescale** value.  

To achieve our counting behaviour, we will set the timer into __CTC__ mode, which stands for __Clear Timer__ on __Compare__ Mode.  **CTC** is the mode of operation where the timer will count up and once a limit is reached (the *compare* becomes true), it will run our Interrupt Service Routine (ISR) and then reset itself.

Read the following commented code, which will be explained in depth next:







```c
// Routine to setupt timer3 to run
void setupTimer3() {
  
  // disable global interrupts
  cli();          

  // Reset timer3 to a blank condition.
  // TCCR = Timer/Counter Control Register
  TCCR3A = 0;     // set entire TCCR3A register to 0
  TCCR3B = 0;     // set entire TCCR3B register to 0

  // First, turn on CTC mode.  Timer3 will count up
  // and create an interrupt on a match to a value.
  // See table 14.4 in manual, it is mode 4.
  TCCR3B = TCCR3B | (1 << WGM32);

  // For a cpu clock precaler of 256:
  // Shift a 1 up to bit CS32 (clock select, timer 3, bit 2)
  // Table 14.5 in manual.
  TCCR3B = TCCR3B | (1 << CS32);
  
  
  // set compare match register to desired timer count.
  // CPU Clock  = 16000000 (16mhz).
  // Prescaler  = 256
  // Timer freq = 16000000/256 = 62500
  // We can think of this as timer3 counting up to 62500 in 1 second.
  // compare match value = 62500 / 2 (we desire 2hz).
  OCR3A = 31250;
  
  // enable timer compare interrupt:
  TIMSK3 = TIMSK3 | (1 << OCIE3A);

  // enable global interrupts:
  sei();
  
}
```

The above code are all the steps required to setup _timer3_ to activate the ISR every 500ms.  

**Yes - it does look very obscure!  We will now explore it.**

The most important parts of this routine are the values stored into the locations **TCCR3B** and **OCR3A**.  

TCCR3B and OCR3A are known as **_registers_**.  A register is an internal part of a computer which acts like switches to turn other parts of the computer on and off.  To use them, we simply write code to store a number in a similar way to using a variable.  

You may notice that no where in the example code are OCR3A, or TIMSK3, or OCIE3A, etc, declared as variables - and yet the compiler will not create an error.  Within the Arduino environment, these capitalised names are **keywords** to the registers.  They are specific references to access the registers as if they were variables you had declared.  As a general rule, the names present in the micro-controller manual will have been used as Arduino keywords.  If in doubt, search for examples on the internet. If you feel more courageous, look into the Arduino environment to find header files (.h).  Unfortunately, this is a piece of knowledge that has little more explanation, and must simply be remembered.  With experience, you'll simply go looking for the source files or help online.

Unlike a normal variable, a register will typically utilise the **binary** value stored, where each individual bit is used to indicate **on**(1) or **off**(0) for a specific device or function within the microcontroller. You can think of most registers as a row of switches.  So it is normal to need to write single bits to registers.  Sometimes, we need to write more than 1 bit to different locations, and so we use binary masks and bitwise operators.  We will explore this in more depth in just a moment.  

For now, the following two registers set up our timer so that:  

- OCR3A stores the value to count up to, using 16 binary bits, in order to determine when to stop the timer and run the ISR.

- TCCR3B stores the prescaler value, set by a combination of 3 individual bits, which determines how quickly your timer will be counting up as a division of the CPU clock.

Therefore, our setup can be described in two steps as:
- __what value to count up to__ (OCR3A)
- __how quickly the timer counts__ (TCCR3B)

When the Timer3 count reaches the match value, the ISR is run, and timer3 resets its own count to zero.  It will count up toward OCR3A again.


<hr><br><br><br><br>

## A closer look at interacting with **registers**

In the above example setupTimer3() code, we used:

```c
  // Reset timer3 to a blank condition.
  // TCCR = Timer/Counter Control Register
  TCCR3A = 0;     // set entire TCCR3A register to 0
  TCCR3B = 0;     // set entire TCCR3B register to 0

  // First, turn on CTC mode.  Timer3 will count up
  // and create an interrupt on a match to a value.
  // See table 14.4 in manual, it is mode 4.
  TCCR3B = TCCR3B | (1 << WGM32);
```


As the comment says, this configures the timer into __CTC__ mode, which stands for __Clear Timer__ on __Compare__ Mode.  We will now look in depth at how this code works.  The following image is a table from the manual which described the possible modes of the timer:


<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/timer3_mode_table.png?raw=true" alt="Timer3 modes, page 133"/>

In the above table extract we can see many possible modes for Timer 3.  To use any one of them, WGMn3, WGMn2, WGMn1, and WGMn0 need to be set with either a 0 (off) or 1 (on).  WGM stands for _Waveform Generation Mode_, which is a little cryptic.  However, if we set up a device to repeatedly turn on and off automatically, it will essentially generate a waveform (a common use is to generate a Pulse Width Modulation waveform), hence the name.

In our example code we started of by setting all bits of the registers TCCR3A and TCCR3B to 0.  This ensures that any previous set up for the timer is cleared out.  Not doing this can cause some very random behaviours - which are some of the most difficult to find bugs in your code.


We can read the table and see that for CTC mode we need to:
 - set WGMn3 to 0
 - set WGMn2 to 1 (circled red, above)
 - set WGMn1 to 0
 - set WGMn0 to 0.

In the key word WGMn3/2/1/0, we will use n = 3, because we are setting up timer3, e.g. WGM**3**0, WGM**3**1, WGM**3**2, WGM**3**3.    

Next, we need to know in which **register** the bits WGMn0/1/2/3 are located.  Even though WGMn0/1/2/3 appear as sequential bits in the table for configuration, it does not mean that they will occur sequentially in the actual registers!  In the manual, we can find that WGM33 and WGM32 are located in the register TCCR3B, and WGM31 and WGM30 are located in TCCR3A - and so thats how and why we end up knowing to use TCCR3A and TCCR3B:

<pre>
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/TCCR3B_register_table.png?raw=true"/>
<br>
<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/TCCR3A_register_table.png?raw=true"/>
</pre>

Note that, WGM32 is located in bit 3 of TCCR3B (counting across right-to-left from 0) of the register.

We only need WGM32 set to 1.  We could write this in code as:

<pre>
 &nbsp;<font color="#000000">TCCR3B</font> <font color="#434f54">=</font> <font color="#000000">0b00001000</font><font color="#000000">;</font>
</pre>

The above code explicitely writes a binary value into the register.  We know it is an explicit binary value because it is indicated by the 0b prefix.  Also notice that in the binary value, only bit 3 (counting across right-to-left starting at 0) is set to 1.

However, writing a register in the above way **is very bad practice** (please don't do it this way on an important project!). Why is it bad?
- Writing a value in this way sets all the bits simultaneously.  Bits 0-2 are set to 0, bit 3 is set to 1, and bits 4-7 are set to 0.  If we had to use the other bits for configuration, they would all be changed with this one line of code.
- If we wanted to run our code on a new microcontroller, there is a good chance the register and the configuration of bits would be different.  Using an explicit binary value will not transfer well to other devices.  Therefore, the whole reason for using these cyptic keywords is to help make your code portable between devices.

The better way to write a bit value into a register is to use **bitwise** operators, and to use **register keywords**.  The following line of code is much better practice:

<pre>
 &nbsp;<font color="#000000">TCCR3B</font> <font color="#434f54">=</font> <font color="#000000">TCCR3B</font> <font color="#434f54">|</font> <font color="#000000">(</font><font color="#000000">1</font> <font color="#434f54">&lt;&lt;</font> <font color="#000000">WGM32</font><font color="#000000">)</font><font color="#000000">;</font>
</pre>

We can read the above line of code in natural language as something like:

- `TCCR3B = ` Store into TCCR3B...
-  ` TCCR3B |` ...the current value of TCCR3B Logically OR'd with...
-  `( 1 << WGM32 )` ...a 1 shifted across to the location of WGM32.

Where:
- _(1 << WGM32)_ means shift a 1 up to the location of WGM32.
- We use the logical OR operation (the | symbol) because it maintains any other values in the register.


If you want more information on manipulating 1's and 0's within registers and variables, <a href="https://playground.arduino.cc/Code/BitMath">look up bitwise operations, masking and logical truth tables</a>.  As a rule of thumb, we use OR to mask in a high value, and we use AND to mask in a low value.  You can also use the Arduino <a href="https://www.arduino.cc/reference/en/language/functions/bits-and-bytes/bitset/">bitSet()</a> and <a href="https://www.arduino.cc/reference/en/language/functions/bits-and-bytes/bitclear/">bitClear()</a> functions.  



Whilst we have only looked at TCCR3B and WGM32 in detail here, it is the same steps and process for the other registers.  For each of the registers in the code above, you can find more information within the <a href="http://ww1.microchip.com/downloads/en/devicedoc/atmel-7766-8-bit-avr-atmega16u4-32u4_datasheet.pdf">manual</a>.




<hr><br><br><br><br>

## Setting a Prescaler value

Setting the prescale value follows the same process as above, but often you will need to set more than one bit.  Knowing which bits to set is again done by referring to the **prescaler table within the manual**:

<img src="https://github.com/paulodowd/EMATM0054_53/blob/main/Images/prescale_table.png?raw=true"/>


The above table shows us that for any one of the prescale values, we will need to set **a combination of** the CS32, CS31 and CS30 bit values.  Again, the n in CSn2/1/0 refers to the timer number, in our case 3.  We have to look in the manual to find out which register CSn2/1/0 are located in.  These are all located within TCCR3B (TCCR of timer3  B register) - the table is shown in the prior section, labeled 14.10.4.  Our current code example uses the line:


<pre>
 &nbsp;<font color="#000000">TCCR3B</font> <font color="#434f54">=</font> <font color="#000000">TCCR3B</font> <font color="#434f54">|</font> <font color="#000000">(</font><font color="#000000">1</font> <font color="#434f54">&lt;&lt;</font> <font color="#000000">CS32</font><font color="#000000">)</font><font color="#000000">;</font>
</pre>

From the table we can see that placing a value of 1 into bit CS32 sets the prescale at 256.  

If we wanted a prescale value of 1024, we would instead use:

<pre>
&nbsp;<font color="#000000">TCCR3B</font> <font color="#434f54">=</font><font color="#000000"> TCCR3B</font> <font color="#434f54">|</font> <font color="#000000">(</font><font color="#000000">1</font> <font color="#434f54">&lt;&lt;</font> <font color="#000000">CS30</font><font color="#000000">)</font> <font color="#434f54">|</font> <font color="#000000">(</font><font color="#000000">1</font> <font color="#434f54">&lt;&lt;</font> <font color="#000000">CS32</font><font color="#000000">)</font><font color="#000000">;</font>
</pre>

Hopefully, you can understand that writing...

<pre>
&nbsp;<font color="#000000">TCCR3B</font> <font color="#434f54">=</font><font color="#000000"> 1024</font><font color="#000000">;</font>
</pre>

... would **not** set the prescaler to a prescale value of 1024, but would set the register bits to the binary representation of 1024.

<br><br><br><br>

# Timer3 Code Template

We are now ready to look at a complete code example using Timer 3!  

Type out the following code to an Arduino sketch, and it will blink the onboard LED at 0.5 second intervals.

```c
// Global variable to remember the
// on/off state of the LED.  
volatile boolean DEBUG_LED_STATE = false;


// Remember: Setup only runs once when the arduino is powered up.
void setup() {
  
  // Enable pin 13 LED for debugging
  pinMode(13,OUTPUT );
  digitalWrite( 13, DEBUG_LED_STATE );
  
  // Start Serial monitor and print "reset"
  // so we know if the board is reseting
  // unexpectedly.
  Serial.begin(9600);
  delay(1500);
  Serial.println("***RESET***");

  // Setup our timer3
  setupTimer3();
}


// Remember: loop is called again and again.
void loop() {

  // Nothing to do here.
  delay(100);
}


// Routine to setupt timer3 to run
void setupTimer3() {
  
  // disable global interrupts
  cli();          

  // Reset timer3 to a blank condition.
  // TCCR = Timer/Counter Control Register
  TCCR3A = 0;     // set entire TCCR3A register to 0
  TCCR3B = 0;     // set entire TCCR3B register to 0

  // First, turn on CTC mode.  Timer3 will count up
  // and create an interrupt on a match to a value.
  // See table 14.4 in manual, it is mode 4.
  TCCR3B = TCCR3B | (1 << WGM32);

  // For a cpu clock precaler of 256:
  // Shift a 1 up to bit CS32 (clock select, timer 3, bit 2)
  // Table 14.5 in manual.
  TCCR3B = TCCR3B | (1 << CS32);
  
  
  // set compare match register to desired timer count.
  // CPU Clock  = 16000000 (16mhz).
  // Prescaler  = 256
  // Timer freq = 16000000/256 = 62500
  // We can think of this as timer3 counting up to 62500 in 1 second.
  // compare match value = 62500 / 2 (we desire 2hz).
  OCR3A = 31250;
  
  // enable timer compare interrupt:
  TIMSK3 = TIMSK3 | (1 << OCIE3A);

  // enable global interrupts:
  sei();
  
}

// The ISR routine.
// The name TIMER3_COMPA_vect is a special flag to the
// compiler.  It automatically associates with Timer3 in
// CTC mode.
ISR( TIMER3_COMPA_vect ) {

  // Invert LED state.
  DEBUG_LED_STATE = !DEBUG_LED_STATE;

  // Enable/disable LED
  digitalWrite(13, DEBUG_LED_STATE);

}
```

<hr><br><br><br><br>

# Exercise 1:
1. Adjust the above setupTimer3() routine to get your LED to flash at 4hz, 40hz, 500hz, and 1khz.
    - Hint: 25hz is the limit of human vision to see flashing.  You may need to use Serial.print() to evaluation your code.
2. The Buzzer is on digital pin 6.  Adapt your code to sound the buzzer instead of flashing the LED.

3. Recreate the millis() functionality.  
  - Set your Timer3 ISR to run at 1khz.  Add an extra global variable, an unsigned long, to your code and get your ISR to increment the value.  
  - Use _Serial.println()_ within loop() to check its progress.  
  - Try reseting the value at intervals.  
  - Try this with and without the _volatile_ keyword added to your variable - what is the observable consequence?





# Exercise 2:

If you have made it this far through the labsheet, you should be able to configure timer3 to be useful to you.  You may wish to look at running a PID updates within the timer3 ISR, or you may wish to regularly update a sensor. Look out for:
- Bugs caused by an ISR that takes too long to execute.  If an ISR is taking too long to execute, your normal thread of code will not have many computation cycles to operate before the interrupt is called again.
- Bugs caused by non-volatile variable access.  When you read from a global volatile memory, it is possible that the read operate is itself interrupted.  This means the underlying value of the variable can change during a read process.  
- An interrupt operating very fast.  Even a very short interrupt, called very frequenty, will leave fewer computation cycles remaining for your main thread of code.
- Different interrupts interrupting each other.  When using Timer3, remember that your encoders will also be causing interrupts within the normal operation of your code.  
